<a href="https://colab.research.google.com/github/wipodev/google_colab/blob/main/OpenAI_Whisper_DotCSV_Modificado_por_WipoDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🗣️ **Whisper** - *Notebook creado por [DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg)*
##*modificado por [WipoDev](https://github.com/wipodev)*

🚨 Ten activada la **Aceleración por hardware** con GPU en `"Entorno de ejecución" > "Cambiar tipo de entorno de ejecución"`

In [ ]:
#@title ⚙️ Ejecutar esta celda para instalar e importar las librerías.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install ffmpeg-python # cambie el paquete ffmpeg por ffmpeg-python para solucionar error en ffmpeg.input

from IPython.display import HTML, Audio
from IPython.display import clear_output
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
from scipy.io.wavfile import write
import numpy as np
import io
import os
import time
import ffmpeg
import whisper

In [29]:
#@title ⚙️ Ejecutar esta celda para cargar el modulo de grabación.
AUDIO_HTML = """
<script>
    var my_div = document.createElement("DIV");
    var recordButton = document.createElement("BUTTON");

    // Establecer estilos en línea para el div
    my_div.style.display = "flex";
    my_div.style.flexDirection = "column";
    my_div.style.gap = "10px";
    my_div.style.alignItems = "center";
    my_div.style.padding = "10px";

    // Establecer estilos en línea para el botón
    recordButton.style.backgroundColor = "red";
    recordButton.style.color = "white";
    recordButton.style.margin = "0 auto";
    recordButton.style.border = "none";
    recordButton.style.borderRadius = "5px";
    recordButton.style.padding = "5px 10px";
    recordButton.style.cursor = "pointer";

    my_div.appendChild(recordButton);
    document.body.appendChild(my_div);

    var base64data = 0;
    var reader;
    var recorder, gumStream;

    var handleSuccess = function (stream) {
      gumStream = stream;
      var options = {
        mimeType: 'audio/webm;codecs=opus'
      };
      recorder = new MediaRecorder(stream);
      recorder.ondataavailable = function (e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        my_div.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function () {
          base64data = reader.result;
        }
      };
      recorder.start();
    };

    recordButton.innerText = "Grabando... presione para detener";

    navigator.mediaDevices.getUserMedia({ audio: true }).then(handleSuccess);


    function toggleRecording() {
      if (recorder && recorder.state == "recording") {
        recordButton.innerText = "Guardando la grabación... ¡espera por favor!"
        recordButton.style.backgroundColor = "yellow";
        recordButton.style.color = "black";
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
      }
    }

    function sleep(ms) {
      return new Promise(resolve => setTimeout(resolve, ms));
    }

    var data = new Promise(resolve => {
      recordButton.onclick = () => {
        toggleRecording()

        sleep(2000).then(() => {
          resolve(base64data.toString())
        });
      }
    });

  </script>
"""

AUDIO_SAVE = """
<script>
recordButton.innerText = "Audio guardado con éxito"
recordButton.style.backgroundColor = "green";
recordButton.style.color = "white";
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Divida el tamaño del fragmento en cuatro bytes, guardados en b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Reemplace los bytes 4:8 en proc.stdout con el tamaño real del fragmento RIFF.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
#@title 🎙️ **Grabar audio**
file_name = 'record.wav' # Nombre del archivo WAV

audio, sr = get_audio()
write(file_name, sr, audio)

# Define un tiempo límite para la verificación (ajusta según tus necesidades)
time_limit = 60  # segundos

# Espera hasta que termine la escritura o se alcance el tiempo límite
initial_time = time.time()
file_exists = False

while time.time() - initial_time < time_limit:
    if os.path.exists(file_name):
        # El archivo existe, verifica si su tamaño ha dejado de cambiar
        previous_size = os.path.getsize(file_name)
        time.sleep(1)  # Espera un segundo
        current_size = os.path.getsize(file_name)

        if current_size == previous_size:
            file_exists = True
            break

if file_exists:
  display(HTML(AUDIO_SAVE))

## 🎙️ **Transcribe o traduce** [Elige la tarea]

In [ ]:
Tarea = "Transcribir" #@param ["Transcribir", "Traducir al inglés"]
Audio = "record" #@param {type:"string"}

task = "translate" if Tarea == "Traducir al inglés" else "transcribe"
output = '/content/'+Audio+'/'+task

!whisper {Audio +".wav"} --task {task} --model medium --verbose False --output_dir {output}

clear_output()
if task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN --\n")

print(open(output+'/record.txt').read())